# **Practico Mentoria - Introduccion al Aprendizaje Automatico**

Se propone la elaboración de un informe o presentación, en formato estatico:
* PDF
* Markdowns
* Google Docs

Que responda a las cuestiones solicitadas en cada seccion de esta **Jupyter Notebook**.

La comunicación debe estar apuntada a un público técnico pero sin conocimiento del tema particular, como por ejemplo, sus compañeros de clase.  
Por lo cual debe estar redactado de forma consisa y comprensible.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, mean_squared_error, confusion_matrix
from sklearn import preprocessing

In [ ]:
filename = "../Introduccion_Mentoria/dataset_inf_telec_20200501T130000_20200727T010000_v1.csv"

In [ ]:
df = pd.read_csv(
    filename,
    dtype={
        'ID_EQUIPO': int,
        'PUNTO_MEDICION': str,
        'CAPACIDAD_MAXIMA_GBS': float,
        'PASO': int,
        'LATENCIA_MS': float,
        'PORCENTAJE_PACK_LOSS': float,
        'INBOUND_BITS': np.float64,
        'OUTBOUND_BITS': np.float64,
        'MEDIDA': str,
    },
    parse_dates=[
        'FECHA_INICIO_MEDICION',
        'FECHA_HORA',
        'FECHA_FIN_MEDICION',
    ],
    na_values=['NaN']
)

In [ ]:
df.shape

In [ ]:
df.sample(5)

In [ ]:
print(df.FECHA_HORA.min())

print(df.FECHA_HORA.max())

---

## **Regresion**

Elija algun `PUNTO MEDICION`, por ejemplo **ABA - Abasto Cliente**.

Vamos a predecir el `LATENCIA_MS` de dicho punto de medición.

In [ ]:
# TODO

In [ ]:
# Separamos el "target" del resto del dataset
X = df.loc[:, df.columns != 'LATENCIA_MS']
y = df['LATENCIA_MS']

Seleccionamos uno o más feature del dataset que no sea categórico, por ejemplo `INBOUND_BITS`

In [ ]:
# TODO: modificar esta feature por algún otro (o una combinacion de estos) para ver como cambian los resultados
X = X[['INBOUND_BITS']]

## **División de datos en conjuntos de entrenamiento y evaluación**

La primer tarea consiste en dividir el conjunto de datos cargados en el apartado anterior en conjuntos de entrenamiento (training) y evaluación (test).

Utilizar aproximadamente 70% de los datos para entrenamiento y 30% para validación.

Links:
* https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
# TODO

>### **Regresion Lineal**

Link:
* https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [ ]:
# TODO

Evaluamos el desempeño del clasificador utilizando la media del error cuadrado (`MSE` o `Mean Squared Error`) sobre el conjunto de datos de entrenamiento `(X_train, y_train)` y lo comparamos con el de validación `(X_val, y_test)`.
Mientras más cercano a cero mejor

In [ ]:
print(f"MSE para entrenamiento: {mean_squared_error(y_train, model.predict(X_train)):.2f}")
print(f"MSE para validación   : {mean_squared_error(y_test, model.predict(X_test)):.2f}")

#### **Visualizacion**

**Warning**: Tener en cuenta que si son dos o mas features no se va a poder visualizar

In [ ]:
plt.figure(figsize=(14, 5), dpi= 80, facecolor='w', edgecolor='k')

X_range_start = np.min(np.r_[X_train, X_test])
X_range_stop = np.max(np.r_[X_train, X_test])
y_range_start = np.min(np.r_[y_train, y_test])
y_range_stop = np.max(np.r_[y_train, y_test])
X_linspace = np.linspace(X_range_start, X_range_stop, 200).reshape(-1, 1)

# Conjunto de entrenamiento
plt.subplot(1, 2, 1)
plt.scatter(X_train, y_train, facecolor="dodgerblue", edgecolor="k", label="datos")
plt.plot(X_linspace, model.predict(X_linspace), color="tomato", label="modelo")
plt.ylim(y_range_start, y_range_stop)
plt.title("Conjunto de Entrenamiento")

# Conjunto de validación
plt.subplot(1, 2, 2)
plt.scatter(X_test, y_test, facecolor="dodgerblue", edgecolor="k", label="datos")
plt.plot(X_linspace, model.predict(X_linspace), color="tomato", label="modelo")
plt.ylim(y_range_start, y_range_stop)
plt.title("Conjunto de Validación")

plt.show()

>### **EXTRA: Regresión Polinomial**

Link:
* https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html

---

## **Clasificacion**

Elija algun `PUNTO MEDICION`, por ejemplo **ABA - Abasto Cliente**.

Vamos a predecir el `PORCENTAJE_PACK_LOSS` de dicho punto de medición.  
Como esta variable no es categorica, vamos a codificarla como tal (guiarse por lo que saben de **Analisis y Curacion de Datos**), para ello vamos a tomar los siguientes rangos:
* Si `PORCENTAJE_PACK_LOSS` $\in [0, 0.05) \Rightarrow$ 0
* Si `PORCENTAJE_PACK_LOSS` $\in [0.05, 0.1) \Rightarrow$ 1
* Si `PORCENTAJE_PACK_LOSS` $\in [0.1, \infty) \Rightarrow$ 2

In [ ]:
# Separamos el "target" del resto del dataset
X = df.loc[:, df.columns != 'PORCENTAJE_PACK_LOSS']
y = df['PORCENTAJE_PACK_LOSS']

In [ ]:
# TODO: Codificar la variable `PORCENTAJE_PACK_LOSS`

Seleccionamos uno o más feature del dataset que no sea categórico, por ejemplo `INBOUND_BITS` y `OUTBOUND_BITS`

In [ ]:
# TODO: modificar esta feature por algún otro (o una combinacion de estos) para ver como cambian los resultados
X = X[['INBOUND_BITS', 'OUTBOUND_BITS']]

## **División de datos en conjuntos de entrenamiento y evaluación**

La primer tarea consiste en dividir el conjunto de datos cargados en el apartado anterior en conjuntos de entrenamiento (training) y evaluación (test).

Utilizar aproximadamente 70% de los datos para entrenamiento y 30% para validación.

Links:
* https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
# TODO

>### **Regresion Logistica**

Link:
* https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [ ]:
penalty = # TODO: Tipo de regularización: l1 (valor absoluto), l2 (cuadrados).
alpha = # TODO: Parámetro de regularización. También denominado como parámetro `lambda`. Debe ser mayor que 0.

model = LogisticRegression(penalty=penalty, C=1./alpha, multi_class='ovr')
model.fit(X_train, y_train)

print(f"Accuracy para entrenamiento: {accuracy_score(y_train, model.predict(X_train)):.2f}")
print(f"Accuracy para validación   : {accuracy_score(y_test, model.predict(X_test)):.2f}")

#### **Matriz de Confusion**

Plotear las matrices de confunsion y obtener algunas conclusiones

In [ ]:
# TODO